In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

In [ ]:
cookies = {
    "user": "en"
    }
req_headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "x-requested-with": "XMLHttpRequest",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache"
    }
proxies = {"https": "socks5://72.223.168.67:4145"}

In [70]:
def crawl_product_id():
    product_list = []
    i = 1
    while (True):
        try:
            # print("Crawl page: ", i)
            url_m = 'https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&aggregations=2&trackity_id=8a6c7f5d-a457-a406-b969-116022428191&category=915&page={}&urlKey=thoi-trang-nam'.format(i)
            url_f = 'https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&aggregations=2&trackity_id=8a6c7f5d-a457-a406-b969-116022428191&category=931&page={}&urlKey=thoi-trang-nu'.format(i)
            response_m = requests.get(url_m, headers=req_headers, cookies=cookies, verify=False)
            response_f = requests.get(url_f, headers=req_headers, cookies=cookies, verify=False)

            data_m = json.loads(response_m.text)['data']
            data_f = json.loads(response_f.text)['data']

            for m, f in zip(data_m, data_f):
                product_list.append(m['id'])
                product_list.append(f['id'])
            i += 1
            if i==101:
                break
        except:
            print('ERORRRRRRRRRRRRRRRRRRRRR')
            break

    return product_list, i


In [ ]:
product_id_list, _ = crawl_product_id()

In [ ]:
revs_dict = dict(content = [], star = [])
for id in product_id_list:
    response_data = requests.request("GET" ,'https://tiki.vn/api/v2/reviews?include=comments&product_id={}&limit=100'.format(id), headers=req_headers, verify=False)
    revs = response_data.json()['data']
    revs = sorted(revs, key=lambda d:d['rating'])
    for r in revs:
        if len(r['content'])>50:
            revs_dict['content'].append(r['content'])
            revs_dict['star'].append(r['rating'])

In [75]:
df = pd.DataFrame(revs_dict)

In [79]:
df.to_csv('datafull.csv', index=False)

In [91]:
one = df[df.star==1]
two = df[df.star==2]
three = df[df.star==3]
four = df[df.star==4]
five = df[df.star==5]

In [94]:
one = one.sample(n=1334)
two = two.sample(n=666)
three = three.sample(n=667)
four = four.sample(n=666)
five = five.sample(n=667)

In [95]:
sam = pd.concat([one, two])
sam = pd.concat([sam, three])
sam = pd.concat([sam, four])
sam = pd.concat([sam, five])

In [97]:
def label(x):
    if x==1: return 'Tiêu cực'
    elif x in [2, 3]: return 'Trung tính'
    elif x in [4, 5]: return 'Tích cực'

In [99]:
sam['label'] = sam['star'].apply(lambda x: label(x))

In [103]:
sam.groupby('star').count()

,content,label
star,,
1,1334,1334
2,666,666
3,667,667
4,666,666
5,667,667


In [104]:
sam.head()

,content,star,label
35454,"combo 3 cái gĩao có 1 cái, thành ra đặt 6 cái...",1,Tiêu cực
11006,mình mua áo có cổ màu trắng lại ship tới cho m...,1,Tiêu cực
47850,Giao sai hàng. Tôi muốn trả hàng. Đặt be đậm g...,1,Tiêu cực
23763,Sản xuất vn nhưng thấy in chữ trung quốc nên k...,1,Tiêu cực
12024,Mình đặt áo sơ mi trắng dài tay mà shop giao c...,1,Tiêu cực
